# Lecture 7

## Parsing with CFGs and PCFGs: the CKY Algorithm

### Two Approaches to CFG Parsing
1. Bottom-up: start with the words (terminal symbols) and see which subtrees you can build, then combine subtrees into larger trees (process is driven by the input sentence)
    * CKY Algorithm - requires grammars to be in Chomsky Normal Form
2. Top-down: start at the start symbol (S), try to apply prouction rules that are compatible with the input (process is driven by the grammar). 
    * Earley Algorithm

### Chomsky Normal Form
* A CFG $G = (N, \Sigma, R, S)$ is in Chomsky Normal Form (CNF) if the rules take one of the following forms:
    * $A \rightarrow B C$ where $A, B, C \in N$
    * $A \rightarrow a$ where $A \in N, b \in \Sigma$

* If a rule derives two variables, they must both be nonterminals
* If a rule derives a single variable, it must be terminal
* Any CFG can be converted to a grammar in CNF that represents the same language

### Cocke-Kasami-Younger (CKY) Algorithm - Motivation
* A nonterminal A covers a sub-span $s[i, j]$ of the input string $s$ if the rules in the grammar can derive $s[i, j]$ from A
* Let $\pi[i, j]$ be the set of nonterminals that cover $[i, j]$

* The string is recognized by the grammar if $S \in \pi[i, j]$

* Approach: compute $\pi[i, j]$ from the bottom-up, using dynamic programming

### CKY - Recursive Definition
* To compute $\pi[i, j]$, try all possible split points $k$ such that $i < k < j$
    * For each $k$, check if the nonterminals in $\pi[i, k]$ and $\pi[k, j]$ match any of the rules in the grammar
    * Update $\pi[i, j]$ to include any new rules found 

### Syntactic Ambiguity
* Multiple parse trees for the same sentence/phrase indicates that the phrase can be interpreted in multiple ways (depending on emphasis)
* Construct parse trees from the CKY Algorithm by storing backpointers, in addition to the nonterminals that derive the variables at every step

### Probabilities for Parse Trees
* In order to select a parse tree (there may be infinitely many generated by a grammar G), we must assign a probability to each parse tree
* Sum of probabilities of all parse trees should be 1


* The most likely parse tree produced by $G$ for string $s$ is:

$$
\arg \max_{t \in T_G(s)} P(t)
$$

### Probabilistic Context Free Grammars (PCFG)
* A PCFG consists of a Context Free Grammar where there is a probability $P(A \rightarrow \beta)$ for each rule
* The probabilities for all rules with the same left-hand-side sum up to 1:

$$
\sum_{\beta} P(A \rightarrow \beta) = 1
$$

sum over all $\beta$ generated by $A$ and the sum should equal 1. 

### Estimating PCFG Probabilities
* Supervised training: we can estimate PCFG probabilities from a *treebank*, a corpus manually annotated with constituency structure using maximum likelihood estimates:

$$ P(A \rightarrow \beta) = \frac{count(A \rightarrow \beta)}{count(A)}
$$

* Can also apply smoothing and other techniques in case any tokens are unseen

### CKY for PCFG Parsing
* Adapt the CKY algorithm to construct the most likely parse tree -- at each step, store the rule that has the highest probability at location i, j

$$
\pi[i, j, X] = \max_{t \in T_G(s[i,j]) : root(t) = X} P(t)
$$

At each location [i,j] and for each rule X, store the rule $X \rightarrow \beta$ if this rule has the greatest probability out of all rules with X on the LHS

* Recursive definition:

Base case: $\pi[i, i + 1, A] = \begin{cases} P(A \rightarrow s_i) & \text{if } A \rightarrow s_i \in R \\
0 & \text{otherwise}
\end{cases}$

$$
\pi[i, j, A] = \max_{k=i + 1, ..., j-1} P(A \rightarrow BC) \cdot \pi[i, k, B] \cdot \pi[k, j, C]
$$

for all $ A \rightarrow BC  \in R$